In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_csv("../data/raw/train.csv")

In [4]:
from src.features.preprocess import preprocess
df["preproc_text"] = df["text"].apply(preprocess)

In [11]:
for row in df.sample(2).itertuples():
    print(row.text)
    print(row.preproc_text)
    print(row.sentiment)
    print("===========")

 ... Waited for the band before the Singapore show, met Chris M and got tickets... Nothing you don`t know!
waited for the band before the singapore show met chris m and got tickets nothing you do not know
neutral
_indian hehe very true  but you`ve got the color advantage, which the complete americans haven`t
hehe very true but you have got the color advantage which the complete americans have not
neutral


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["preproc_text"])

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df["sentiment"])

In [33]:
y, df["sentiment"]

(array([2, 1, 0, ..., 2, 0, 2]),
 0        positive
 1         neutral
 2        negative
 3        negative
 4        negative
            ...   
 24727    negative
 24728     neutral
 24729    positive
 24730    negative
 24731    positive
 Name: sentiment, Length: 24732, dtype: object)

In [37]:
le.

{}

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)

SVC()

In [22]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_val)
accuracy_score(y_val, y_pred)

0.6844552253891247

In [23]:
test_df = pd.read_csv("../data/raw/test.csv")

In [24]:
X_test =  vectorizer.transform(test_df["text"].apply(preprocess))

In [25]:
y_test_pred =clf.predict(X_test)

In [38]:
target_conversion = {
    0: -1,
    1: 0,
    2: 1
}
submission_df = pd.DataFrame({"textID": test_df["textID"], "sentiment": y_test_pred})
submission_df["sentiment"] = submission_df['sentiment'].map(target_conversion)
submission_df

,textID,sentiment
0,102f98e5e2,1
1,033b399113,-1
2,c125e29be2,1
3,b91e2b0679,-1
4,1a46141274,1
...,...,...
2743,0bfb1006b9,-1
2744,f1df499466,-1
2745,de4da367a4,0
2746,07a8ec4593,1


In [39]:
submission_df.to_csv("submission_svm.csv", index=False)

In [40]:
!kaggle competitions submit -c eurecom-aml-2022-challenge-3 -f submission_svm.csv -m "SVM + tf-idf"

100%|██████████████████████████████████████| 35.4k/35.4k [00:00<00:00, 71.7kB/s]
Successfully submitted to EURECOM AML 2022:: Challenge 3